In [10]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from sklearn.metrics import accuracy_score
import random

# Если возможно, пытаемся запустить на видеокарте
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Загрузка данных
data_path = "data/labeled.csv"
df = pd.read_csv(data_path)

# Используем токенизатор Hugging Face
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")

# Максимальная длина последовательности и размер батч
MAX_LEN = 128
BATCH_SIZE = 32

# Кастомный датасет
class CustomDataSet(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        comment = row["comment"]
        label = torch.tensor(row["toxic"], dtype=torch.long)  # Для BERT метки должны быть типа long

        # Токенизация текста
        encoding = self.tokenizer(
            comment,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt",
        )

        input_ids = encoding["input_ids"].squeeze(0)
        attention_mask = encoding["attention_mask"].squeeze(0)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "label": label,
        }

new_size = 0.2
reduced_df = df.sample(frac=new_size, random_state=42).reset_index(drop=True)

# Создаем датасет
dataset = CustomDataSet(reduced_df, tokenizer, MAX_LEN)

# Разделяем на обучающую и тестовую выборки 80 / 20 используя random_split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Используем DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Используем предобученную модель BERT для классификации (1 класс — токсичность)
model = AutoModelForSequenceClassification.from_pretrained(
    "ai-forever/ruBert-base", num_labels=2  # Для бинарной классификации (0 или 1)
).to(device)

# Оптимизатор и функция потерь
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

# Функция для обучения модели
def train_model(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0

    # Общее количество батчей в даталоадере
    total_batches = len(dataloader)

    for batch_idx, batch in enumerate(dataloader):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        # Прямой проход через модель
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss  # Loss возвращается автоматически в BertForSequenceClassification
        logits = outputs.logits

        # Обратное распространение ошибки
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Выводим количество оставшихся батчей
        remaining_batches = total_batches - (batch_idx + 1)
        print(f"Batch {batch_idx + 1}/{total_batches} completed. Remaining batches: {remaining_batches}")

    return total_loss / len(dataloader)


# Основной цикл обучения и тестирования
EPOCHS = 1

for epoch in range(EPOCHS):
    print("started training")
    train_loss = train_model(model, train_loader, optimizer, criterion)
    print("initicated train_model()")
    # val_accuracy = evaluate_model(model, test_loader)
    print("initicated evaluate_model()")

    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}")
    # print(f"Validation Accuracy: {val_accuracy:.4f}")


Using device: cpu


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


started training
Batch 1/36 completed. Remaining batches: 35
Batch 2/36 completed. Remaining batches: 34
Batch 3/36 completed. Remaining batches: 33
Batch 4/36 completed. Remaining batches: 32
Batch 5/36 completed. Remaining batches: 31
Batch 6/36 completed. Remaining batches: 30
Batch 7/36 completed. Remaining batches: 29
Batch 8/36 completed. Remaining batches: 28
Batch 9/36 completed. Remaining batches: 27
Batch 10/36 completed. Remaining batches: 26
Batch 11/36 completed. Remaining batches: 25
Batch 12/36 completed. Remaining batches: 24
Batch 13/36 completed. Remaining batches: 23
Batch 14/36 completed. Remaining batches: 22
Batch 15/36 completed. Remaining batches: 21
Batch 16/36 completed. Remaining batches: 20
Batch 17/36 completed. Remaining batches: 19
Batch 18/36 completed. Remaining batches: 18
Batch 19/36 completed. Remaining batches: 17
Batch 20/36 completed. Remaining batches: 16
Batch 21/36 completed. Remaining batches: 15
Batch 22/36 completed. Remaining batches: 14
Ba

In [56]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_curve, auc
import torch

# Функция для тестирования модели с дополнительными метриками
def evaluate_model(model, dataloader):
    model.eval()
    predictions, true_labels = [], []
    probabilities = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Получаем вероятности классов с помощью softmax
            probs = torch.softmax(logits, dim=1).cpu().numpy()
            probabilities.extend(probs[:, 1])  # Вероятности для положительного класса (1)

            # Прогнозы — это индексы максимального значения логитов
            predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Метрика Accuracy
    accuracy = accuracy_score(true_labels, predictions)

    # Метрика F1
    f1 = f1_score(true_labels, predictions)

    # Метрика ROC-AUC (для бинарной классификации)
    try:
        roc_auc = roc_auc_score(true_labels, probabilities)
    except ValueError:
        roc_auc = None  # Если только один класс в данных, ROC-AUC нельзя вычислить

    # Метрика PR-AUC (Precision-Recall AUC)
    precision, recall, _ = precision_recall_curve(true_labels, probabilities)
    pr_auc = auc(recall, precision)

    # Возвращаем все метрики
    return {
        "accuracy": accuracy,
        "f1": f1,
        "roc_auc": roc_auc,
        "pr_auc": pr_auc
    }
metrics = evaluate_model(model, test_loader)

In [57]:
RNN_metrics = {
    'accuracy': 0.8293,
    'f1': 0.7270,
    'roc_auc': 0.8820,
    'pr_auc': 0.8204}

for x in metrics.keys():
    print(f"{x}(BERT): {metrics[x]:.3f}. RNN: {RNN_metrics[x]:.3f}. Разница  = {(metrics[x]/RNN_metrics[x]-1)*100:.2f}%")

accuracy(BERT): 0.924. RNN: 0.829. Разница  = 11.40%
f1(BERT): 0.875. RNN: 0.727. Разница  = 20.36%
roc_auc(BERT): 0.975. RNN: 0.882. Разница  = 10.58%
pr_auc(BERT): 0.942. RNN: 0.820. Разница  = 14.81%


In [35]:
# Проверка работы на случайных данных
def predict_random_sample(model, dataset, amount=5):
    model.eval()

    for i in range(amount):
        sample_idx = random.randint(0, len(dataset) - 1)
        sample = dataset[sample_idx]

        input_ids = sample["input_ids"].unsqueeze(0).to(device)
        attention_mask = sample["attention_mask"].unsqueeze(0).to(device)

        with torch.no_grad():
            # Получаем logits из модели
            output = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = output.logits  # логиты модели (размерность: [1, 2])

            # Преобразуем логиты в вероятности
            probabilities = torch.softmax(logits, dim=-1)  # размерность: [1, 2]
            probability_toxic = probabilities[0][1].item()  # вероятность для класса "Токсичный"

        prediction = "Токсичный" if probability_toxic > 0.5 else "Не токсичный"
        print()
        print(f"Настоящий тип: {'Токсичный' if df.iloc[sample_idx]['toxic'] == 1.0 else 'Не токсичный'} | Предсказание: {prediction}")
        print(f"Комментарий: {df.iloc[sample_idx]['comment']}")
        print()




# Проверка работы на заданных вручную данных:
def predict_manual_input(_model, _tokenizer, text, device):

    # Ввод данных от пользователя
    columns = ["comment", "toxic"]
    data = [[text, 0]]

    # Создание DataFrame
    _df = pd.DataFrame(data, columns=columns)

    sample_idx = 0

    _dataset = CustomDataSet(_df, _tokenizer, MAX_LEN)
    sample = _dataset[sample_idx]

    input_ids = sample["input_ids"].unsqueeze(0).to(device)
    attention_mask = sample["attention_mask"].unsqueeze(0).to(device)

    with torch.no_grad():
        # Получаем logits из SequenceClassifierOutput
        output = _model(input_ids=input_ids, attention_mask=attention_mask)
        logits = output.logits  # логиты модели
        probabilities = torch.softmax(logits, dim=-1)  # размерность: [1, 2]
        probability_toxic = probabilities[0][1].item()  # вероятность для класса "Токсичный"

    prediction = "Токсичный" if probability_toxic > 0.5 else "Не токсичный"
    print()
    print(f"Предсказание: {prediction}")
    print(f"Комментарий: {_df.iloc[sample_idx]['comment']}")


def predict_programm(_model, _tokenizer, device):
    while True:
        q = str(input())
        if q in ["q", "quit", ""]:
            break
        else:
            predict_manual_input(_model, _tokenizer, q, device)

In [44]:
predict_random_sample(model, dataset, 2)


Настоящий тип: Токсичный | Предсказание: Токсичный
Комментарий: БМВизм головного мозга



Настоящий тип: Не токсичный | Предсказание: Не токсичный
Комментарий: какие ваши доказательства !?



In [58]:
predict_programm(model, tokenizer, device)